In [54]:
# pain gaming :/

import pandas as pd
import numpy as np 
import collections 
from sklearn.neighbors import NearestNeighbors

file_path1 = r'C:\Users\chris\Git Hub Repositories\RandomCode\big_data_351\ml-100k\u1.base'
file_path2 = r'C:\Users\chris\Git Hub Repositories\RandomCode\big_data_351\ml-100k\u1.test'
training = pd.read_table(file_path1, header = None)
testing = pd.read_table(file_path2, header=None)
training.columns = ['user_id', 'item_id', 'rating', 'timestamp']
testing.columns = ['user_id', 'item_id', 'rating', 'timestamp']

print(training.shape)
print(testing.shape)


(80000, 4)
(20000, 4)


In [55]:
training = training.drop('timestamp', axis=1)
testing = testing.drop('timestamp', axis=1)

In [56]:
print('The unique ratings are', sorted(training['rating'].unique()))
print('The unique ratings are', sorted(testing['rating'].unique()))

print(training.isna().sum())
print(testing.isna().sum())



The unique ratings are [1, 2, 3, 4, 5]
The unique ratings are [1, 2, 3, 4, 5]
user_id    0
item_id    0
rating     0
dtype: int64
user_id    0
item_id    0
rating     0
dtype: int64


In [63]:
user_matrix_train = training.pivot(index='user_id', columns = 'item_id', values = 'rating').fillna(0)
user_matrix_test = testing.pivot(index='user_id', columns = 'item_id', values = 'rating').fillna(0)
print(user_matrix_train.shape)
print(user_matrix_test.shape)


(943, 1650)
(459, 1410)


In [84]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_matrix_train)


In [86]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(user_matrix_train)
k = 10


In [93]:
input = 1
user_index = input -1

distances, indices = knn.kneighbors(user_matrix_train.iloc[user_index, :].values.reshape(1, -1), n_neighbors = k)

for i in range(1, len(distances.flatten())):
    print('{0}: User {1}: distance {2}'.format(i, user_matrix_train.index[indices.flatten()[i]], distances.flatten()[i]))

1: User 823: distance 0.5942714921431335
2: User 514: distance 0.6051806198568939
3: User 864: distance 0.6182083336186182
4: User 592: distance 0.6282865762239555
5: User 521: distance 0.6322421003918934
6: User 727: distance 0.6322528075400943
7: User 889: distance 0.6332368238640443
8: User 606: distance 0.6350963462175611
9: User 622: distance 0.6359585689756575


In [114]:
indices.flatten()

array([  0, 822, 513, 863, 591, 520, 726, 888, 605, 621], dtype=int64)

In [117]:
user_matrix_train.iloc[822][8]

5.0

In [ ]:
mse = 0


for index, row in testing.iterrows():
    movie = row['item_id']
    distances, indices = knn.kneighbors(user_matrix_train.iloc[row['user_id'], :].values.reshape(1, -1), n_neighbors = k)
    neighbor_ratings_sum = 0
    counter = 0
    for neighbor in indices.flatten():
        try: 
            their_rating = user_matrix_train.iloc[row['user_id']][movie]
            neighbor_ratings_sum += their_rating
            counter += 1 
        except:
            pass
    if counter != 0:
        avg = neighbor_ratings_sum/counter
        difference_local = (row['rating'] - avg)**2
        mse = mse + difference_local
    else:
        mse = mse + (row['rating'])**2

adjusted_mse = 1/20000 * mse 
        


User-based collaborative filtering, a method used in recommendation systems, has its distinct strengths and drawbacks. Among its primary advantages are personalization, community wisdom, and the ability to function without item metadata. It personalizes recommendations by considering the preferences of users with similar tastes, thereby leveraging the collective wisdom of the community. This method does not require detailed information about the items themselves, making it versatile in situations where such data is scarce or unavailable.

However, this approach also faces significant challenges. One of the key drawbacks is its struggle with scalability, especially in large datasets where calculating and maintaining user similarities becomes computationally intensive. Another major limitation is the cold start problem, where the system cannot effectively provide recommendations for new users due to a lack of their historical data. Additionally, user-based collaborative filtering often exhibits a popularity bias, tending to recommend items that are popular among users but potentially overlooking less common items that might be of interest.

In essence, while user-based collaborative filtering excels in providing personalized recommendations and capitalizes on user ratings without needing item data, it is constrained by issues related to scalability, the new user problem, and a tendency to favor popular items. These factors need to be carefully considered when implementing this model in a real-world context.